## This notebook loads the clusterd sequences (resutls from "Read_and_cluster.ipynb") and merges data of chosen behavioral epochs (merge sampling, outward, reward) in order to  perform furthur repaly analysis using bayesina decoder.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from os.path import join
from scipy.ndimage import gaussian_filter1d
from scipy.special import factorial
from scipy.stats import spearmanr
import pftools as pf
import configparser
import os
import matplotlib as mpl
# Initialize the configuration parser
config = configparser.ConfigParser()

# Read the configuration file
#config.read('config.ini')

# Get the data folder path from the config file
#savefolder= config['paths']['savefolder']
plt.rcParams.update({'font.size': 16})
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['xtick.labelsize'] = 16  # You can adjust the font size as needed
plt.rcParams['ytick.labelsize'] = 16  # You can adjust the font size as needed
import mtools as mot
import pickle
from scipy.stats import mode
from scipy.stats import pearsonr

import matplotlib.pyplot as plt

Illustration of the use of these functions
 Mask indexes (burst array indexes): 
 [2, 10, 50] 
Mask indexes expanded to its 2 neighbours: 
 [ 0  1  2  3  4  8  9 10 11 12 48 49 50 51 52] 
Input indexes (array indexes of the positions y or spike positions ysp)
 [3, 7, 12, 30, 51, 100] 
Input indexes excluding those in the set of mask indexes
 [  7  30 100]


In [ ]:
import configparser
import os
from pathlib import Path
onfig = configparser.ConfigParser()
# Detect base directory
try:
    BASE_DIR = Path(__file__).resolve().parent  # For scripts
except NameError:
    BASE_DIR = Path.cwd()  # For Jupyter Notebooks

# Move one level up
BASE_DIR = BASE_DIR.parent

# Load config.ini
config = configparser.ConfigParser()
config.read(os.path.join(os.getcwd(), "config.ini"))

# Construct full paths using pathlib and ensure they end with a separator
skeletonsfolder = str((BASE_DIR / config["paths"]["skeletonsfolder"]).resolve()) + os.sep
savefolder = str((BASE_DIR / config["paths"]["savefolder"]).resolve()) + os.sep
datafolder = str((BASE_DIR / config["paths"]["datafolder"]).resolve()) + os.sep

# Print paths to verify
print("Base Directory:", BASE_DIR)
print("Skeletons Folder:", skeletonsfolder)
print("Save Folder:", savefolder)
print("Data Folder:", datafolder)

In [2]:
cond_names = {
    'sampling_L': 0,
    'sampling_R': 1,
    'outward_L': 2,
    'outward_R': 3,
    'reward_L': 4,
    'reward_R': 5,
    'inward_L': 6,
    'inward_R': 7
}

In [ ]:
## to get the merged data of sampling, outward and reward for burst replay analysis 
cell_types='All'
descriotor='No_chunk_0.5s_transients'
for fol in ['478','481','483','485']:


    filename = fol+'Sesseion_info_All AllNo_chunk_0.5s_transients'


    data_all_tasks = np.load(join(savefolder, filename), allow_pickle=True)
    sess_info= data_all_tasks['sess_info']
    #Masks= data_all_tasks['Masks']# this mask does not include mask of correct trials of single cells


    filename_mask = fol+'Mask_with_correct_cells'
    Masks_ = np.load(join(savefolder, filename_mask), allow_pickle=True)# i added the correct masks for single cell data recently
    Masks=Masks_[0]


    #data_info=np.load(savefolder+fol+'data_all_sessions'+cell_types+'_'+ descriotor+'newodrer', allow_pickle=True)
    session_mask=Masks_[1]







    speed_thr=0
    # add the mask for speed. speed threshold is set to 5.
    smooth_speed=mot.smooth_signal(sess_info['speed'], 13)# moving average over .5  second
    Masks['speed_seq']=np.asarray(smooth_speed)[sess_info['id_peaks']]>speed_thr
    Masks['speed']=smooth_speed>speed_thr


    # select the data you need. Sessen number, epeoch, .... Selected data will be saved with corresposing name. Inward and outward data will be used by place filed code. 
    sessin_numbers=np.arange(len(session_mask))#[4,5,6,7][0,1,2,3]##np.arange(len(session_mask))
    #sessin_numbers=[6]
    #sessin_numbers=[0,1,2,3,4,5,6,7] # soecify which session you want to work with
    #celid=23# number of sample cell to show in the plot

    #cond_number=[3] # conditon name (outwards)
    #cond_number=[8,9,10,11] # inwards
    #cond_number=[0,1] # sampling
    #cond_number=[6,7] # reward
    for con_number in ([3],):#([1,3,5],[0,2,4],[0,2,4,1,3,5]):# to save only otward data use 2 and 3 for left and right runs
            
        trial_type=1 # 1 is correct       0 is failed

        if trial_type==1:
            type_name='correct'
        elif trial_type==0:
            type_name='failed'



        phase=None#None#None#,1 # 0 is learning,    1 is learned,  None is both
        if phase==None:
            phase_name='all'
        elif phase==0:
            phase_name='learning'
        elif phase==1:
            phase_name='learned'
                


        # odd_even=1# even trials
        # run_data_e=mot.apply_masks(sess_info,Masks,cond_number,cond_names,sessin_numbers,odd_even,session_mask,trial_type,phase)
        # #plt.title(run_data_e['sess_name']+' (even) ('+ run_data_e['phase_name']+') ('+run_data_e['cond_name'] +')')

        # odd_even=0# odd trials
        # run_data_o=mot.apply_masks(sess_info,Masks,cond_number,cond_names,sessin_numbers,odd_even,session_mask,trial_type,phase)
        # #plt.title(run_data_e['sess_name']+' (odd) ('+ run_data_e['phase_name']+') ('+run_data_e['cond_name'] +')')

        odd_even=None# all trials
        run_data_all=mot.apply_masks_test(sess_info,Masks,con_number,cond_names,sessin_numbers,odd_even,session_mask,trial_type,phase)



        name_of_conds=' '.join([xn for xn,x in cond_names.items() if x in con_number])

        run_data_all['template']=sess_info['template']
        # with open(savefolder+fol+animal_direction+'_'+ phase_name+'_'+type_name+'_'+descriotor+'_'+cell_types+'test_neworder_with_templates','wb') as f:
        #     pickle.dump(dict(run_data_all), f)   
        #plt.title(run_data_e['sess_name']+' (all) ('+ run_data_e['phase_name']+') ('+run_data_e['cond_name'] +')')
        with open(savefolder+fol+name_of_conds+'_'+ phase_name+'_'+type_name,'wb') as f:
            pickle.dump(dict(run_data_all), f) 